In [2]:
import pandas as pd
from tqdm import tqdm
import math
import matplotlib.pyplot as plt
import datetime
from dateutil.relativedelta import relativedelta
from pandas.tseries.offsets import DateOffset

In [49]:
sue_df = pd.read_csv('data/sue.csv')
permno_list = list((sue_df['OFTIC'].dropna().unique()))
with open("data/sue_tickers.txt", "w") as outfile:
    outfile.write("\n".join(str(permno) for permno in permno_list))

In [4]:
sue_df = pd.read_pickle('data/checkpoint_data/sue_df_linked.pkl')
with open("data/ibes_tickers.txt", "w") as outfile:
    outfile.write("\n".join(str(ibes_ticker) for ibes_ticker in list(sue_df['OFTIC'].unique())))

In [6]:
ret_df = pd.read_csv('data/crsp_dailydata.csv', parse_dates=['date'], usecols=[0, 1, 4])
ret_df = ret_df.dropna()

In [8]:
ret_df.columns = ['PERMNO', 'date', 'PRC']

In [9]:
ret_df_forward = ret_df[['PERMNO', 'date', 'PRC']]
ret_df['forward_date_3m'] = ret_df['date'] + pd.tseries.offsets.BusinessDay(n=61)
ret_df['forward_date_1d'] = ret_df['date'] + pd.tseries.offsets.BusinessDay(n=1)

<ipython-input-9-fa20c4264c85>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ret_df['forward_date_3m'] = ret_df['date'] + pd.tseries.offsets.BusinessDay(n=61)
<ipython-input-9-fa20c4264c85>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ret_df['forward_date_1d'] = ret_df['date'] + pd.tseries.offsets.BusinessDay(n=1)


In [10]:
ret_df_forward_list = {permno: permno_df for permno, permno_df in ret_df_forward.groupby('PERMNO')}
del ret_df_forward

ret_df_list = [[permno, permno_df] for permno, permno_df in ret_df.groupby('PERMNO')]
del ret_df

In [11]:
ret_df_merged_lists = []
for permno, ret_df_subset in tqdm(ret_df_list): 
    ret_df_forward_subset = ret_df_forward_list[permno]
    ret_df_merged = pd.merge_asof(ret_df_subset, 
                        ret_df_forward_subset, 
                        left_on='forward_date_3m', 
                        right_on='date', 
                        suffixes=['', '_3m'], 
                        direction='forward', 
                        tolerance=pd.Timedelta(days=5))
    ret_df_merged_lists.append(ret_df_merged)
ret_df = pd.concat(ret_df_merged_lists).drop(columns=['date_3m'])

100%|███████████████████████████████████████████████████████████████████████████| 16458/16458 [01:55<00:00, 142.32it/s]


In [12]:
ret_df_list = [[permno, permno_df] for permno, permno_df in ret_df.groupby('PERMNO')]
del ret_df

ret_df_merged_lists = []
for permno, ret_df_subset in tqdm(ret_df_list): 
    ret_df_forward_subset = ret_df_forward_list[permno]
    ret_df_merged = pd.merge_asof(ret_df_subset, 
                        ret_df_forward_subset, 
                        left_on='forward_date_1d', 
                        right_on='date', 
                        suffixes=['', '_1d'], 
                        direction='forward', 
                        tolerance=pd.Timedelta(days=5))
    ret_df_merged_lists.append(ret_df_merged)
ret_df = pd.concat(ret_df_merged_lists).drop(columns=['date_1d'])

100%|███████████████████████████████████████████████████████████████████████████| 16458/16458 [01:57<00:00, 139.96it/s]


In [13]:
ret_df['3m_ret'] = ret_df['PRC_3m']/ret_df['PRC_1d'] - 1
ret_df['1d_ret'] = ret_df['PRC_1d']/ret_df['PRC'] - 1

In [14]:
ret_df.to_pickle('data/checkpoint_data/crsp_dailydata_with_period_ret.pkl')